In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats
from scipy.optimize import minimize

import autograd.numpy as auto_np
from autograd import hessian

from utils import is_positive_definite, flat_sigma_to_sigma_matrix, sigma_matrix_to_flat_sigma
from init import init

In [10]:
beta, sigma_flat, s, w, x, y, z, g_0, G_0, beta_0_init, B_0 = init()
gamma, beta_0, beta_1 = beta[:5], beta[5:9], beta[9:]

In [3]:
def likelyhood_ys(w, x, s, z, gamma, beta_0, beta_1, sigma):
    
    prod = 1
    for i in range(len(z)):
        treatment = s[i]
        if treatment == 0:
            prod *= (stats.norm.pdf(z[0][i],  np.dot(x[i],beta_0), sigma[1][1]**2) *
                     stats.norm.cdf((np.dot(-w[i],gamma) - (sigma[0][1]/sigma[1][1]) * 
                    (z[0][i] - np.dot(x[i], beta_0)))/(1-sigma[0][1]**2/sigma[1][1])**0.5))
        else:
            prod *= (stats.norm.pdf(z[1][i],  np.dot(x[i],beta_1), sigma[2][2]**2) *
                     stats.norm.cdf((np.dot(-w[i],gamma) - (sigma[0][2]/sigma[2][2]) * 
                    (z[1][i] - np.dot(x[i], beta_1)))/(1-sigma[0][2]**2/sigma[2][2])**0.5))
    return prod

def g_sigma(sigma, z, w,  x, beta_0, beta_1, gamma, g_0, G_0):

    first_term = likelyhood_ys(w, x, s, z, gamma, beta_0, beta_1, sigma) # likelihood of ys given sigma
    second_term = stats.multivariate_normal.pdf(sigma, g_0, G_0) # density of sigma given g_0, G_0
    third_term = is_positive_definite(sigma) 

    return first_term * second_term * third_term

def q_sigma(sigma, z0, z1,w,  x0, x1, beta_0, beta_1, gamma, g_0, G_0, density=False):

    def sub_log_g_sigma(sigma):
        return np.log(g_sigma(sigma, z, w,  x, gamma, beta_0, beta_1, g_0, G_0))

    H = hessian(sub_log_g_sigma)(sigma)
    V = np.linalg.inv(-H)

    # Compute mu the mode of sub_log_g_sigma
    result = minimize(-sub_log_g_sigma, x0=sigma)
    mode = result.x[0]

    if density:
        # return density of sigma given mode and V
        return stats.multivariate_t.pdf(sigma, mode, V)
    else:
        # sample from multivariate t density
        return stats.multivariate_t.rvs(loc=mode, shape=V)

def sample_sigma(old_sigma, z0, z1,w,  x0, x1, beta_0, beta_1, gamma, g_0, G_0):
    old_likelihood = (g_sigma(old_sigma, z0, z1,w,  x0, x1, beta_0, beta_1, gamma, g_0, G_0)*
                      q_sigma(old_sigma, z0, z1,w,  x0, x1, beta_0, beta_1, gamma, g_0, G_0, density=True))

    sigma = q_sigma(old_sigma, z0, z1,w,  x0, x1, beta_0, beta_1, gamma, g_0, G_0, density=False)

    new_likelihood = (g_sigma(sigma, z0, z1,w,  x0, x1, beta_0, beta_1, gamma, g_0, G_0)*
                    q_sigma(sigma, z0, z1,w,  x0, x1, beta_0, beta_1, gamma, g_0, G_0, density=True))
    acceptance_ratio = new_likelihood / old_likelihood
    if acceptance_ratio > np.random.uniform():
        return sigma
    else:
        return old_sigma

    
def sample_zi_star(w, x, s, z, gamma, beta_0, beta_1, sigma): #zi_star = si_star, z_i(1 - si)
    si_star = np.zeros(len(s))
    for i in range(len(s)):
        treatment = s[i]
        if treatment == 0:
            si_star[i] = stats.truncnorm.rvs(a=-np.inf,b=0, loc=np.dot(w[i], gamma), scale=1)
            z[1][i] = stats.norm.rvs(loc=np.dot(x[i], beta_1), scale=sigma[2][2]**2)
        else:
            si_star[i] = stats.truncnorm.rvs(a=0,b=np.inf, loc=np.dot(w[i], gamma), scale=1)
            z[0][i] = stats.norm.rvs(loc=np.dot(x[i], beta_0), scale=sigma[1][1]**2)
    return si_star, z

def sample_beta(w, x, s, z, gamma, beta_0, beta_1, sigma, B_0, beta_0_init, s_star):
    nouveau_z_a_remplacer = np.zeros(len(s), 3)
    nouveau_z_a_remplacer[:,0], nouveau_z_a_remplacer[:,1], nouveau_z_a_remplacer[:,2] = s_star, z[0], z[1]
    grand_x_a_remplacer = np.zeros(len(s), 3)
    grand_x_a_remplacer[:,0], grand_x_a_remplacer[:,1], grand_x_a_remplacer[:,2] = s, x[:,0], x[:,1] # TODO : à refaire

    sigma_inv = np.linalg.inv(sigma)
    B_0_inv = np.linalg.inv(B_0)
    B = np.linalg.inv(B_0_inv + np.sum([np.dot(np.dot(x[i], sigma_inv), x[i]) for i in range(len(s))]))
    beta_hat = np.dot(B, np.dot(B_0_inv, beta_0_init) +
                      np.sum([np.dot(np.dot(x[i], sigma_inv), nouveau_z_a_remplacer[i]) for i in range(len(s))]))

    new_beta = stats.multivariate_normal.rvs(loc=beta_hat, shape=B)
    return new_beta

In [ ]:
def main_gaussian(w, x, s, z, gamma, beta_0, beta_1, sigma):

    for i in range(10_000):
        sigma = sample_sigma(sigma, z, w, x, beta_0, beta_1, gamma, g_0, G_0)
        s_star, z = sample_zi_star(w, x, s, z, gamma, beta_0, beta_1, sigma)
        beta = sample_beta(w, x, s, z, gamma, beta_0, beta_1, sigma, B_0, beta_0_init, s_star)
    
    return beta, sigma

## Mixture

In [ ]:
m=2
p=[0.5, 0.5]